In [ ]:
!pip install yfinance
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("default")
%matplotlib inline
import yfinance as yf

In [ ]:
class Symbol():
  def __init__(self, dataframe):
    self.data = dataframe
    self.trans_data = self.transfer(self.data)
    self.uptrend = False
    self.downtrend = False
    self.buy = False
    self.sell = False
    self.price = 0
    self.trend(self.trans_data)
  def transfer(self, dataframe):
    f =  dataframe["Close"].to_frame()
    f["SMA30"] = f["Close"].rolling(30).mean()
    f["SMA50"] = f["Close"].rolling(50).mean()
    f["SMA100"] = f["Close"].rolling(100).mean()
    f["SMA200"] = f["Close"].rolling(200).mean()
    f.dropna(inplace = True)
    return f
  def trend(self, trans_data):
    last_price = trans_data.tail(1)
    self.uptrend = last_price["SMA50"].item() > last_price["SMA100"].item() and last_price["SMA100"].item() > last_price["SMA200"].item()
    self.downtrend = last_price["SMA50"].item() < last_price["SMA100"].item() and last_price["SMA100"].item() > last_price["SMA200"].item()
    self.price = last_price["Close"].item()
    self.buy = self.price > last_price["SMA200"].item() and self.price < last_price["SMA50"].item()
    self.sell = self.price < last_price["SMA200"].item() and self.price > last_price["SMA50"].item()
  def plot(self):
    self.trans_data.plot()

In [ ]:
def main(symbol):
  daily = yf.download(tickers = symbol, period = "2Y", interval = "1D")
  daily
  hour = yf.download(tickers = symbol, period = "20D", interval = "60m")
  hour
  fifteen = yf.download(tickers = symbol, period = "5D", interval = "15m")
  fifteen
  daily_dt = Symbol(daily)
  hour_dt = Symbol(hour)
  fif_dt = Symbol(fifteen)
  buy =  daily_dt.uptrend and fif_dt.uptrend and fif_dt.buy
  sell = daily_dt.downtrend and fif_dt.downtrend and fif_dt.sell
  if buy:
    return {
        "name": symbol,
        "signal": "buy",
        "price": fif_dt.price,
    }
  elif sell:
    return {
        "name": symbol,
        "signal": "sell",
        "price": fif_dt.price,
    }
  else:
    return {
        "name": symbol,
        "signal": "none",
        "price": fif_dt.price,
    }

In [ ]:
symbols_list = ["GBPUSD=X", "EURUSD=X", "GC=F", "JPY=X", "AUDUSD=X", "NZDUSD=X", "EURJPY=X","GBPJPY=X", "EURGBP=X", "EURCAD=X"]

In [ ]:
import pandas as pd

# Create an empty DataFrame
df = pd.DataFrame()

# Loop through the symbols list
for s in symbols_list:
  # Get the signal for the current symbol
  signal = main(s)
  # Add the signal to the DataFrame
  df[s] = signal

# Transpose the DataFrame to switch rows and columns
df = df.transpose()

# Print the DataFrame
print(df)

In [ ]:
!pip install flask pandas

In [ ]:
from flask import Flask, render_template_string
import pandas as pd

app = Flask(__name__)

@app.route('/')
def show_dataframe():
    # Assuming `dataframe` is your DataFrame
    html_data = dataframe.to_html()
    return render_template_string("<html><body>{{table|safe}}</body></html>", table=html_data)

if __name__ == "__main__":
    app.run(debug=True)


In [ ]:
full_m15 = yf.download(tickers = "GC=F", period = "50D", interval = "15m")

In [ ]:
full_m15

In [ ]:
# @title High vs Low

from matplotlib import pyplot as plt
full_m15.plot(kind='scatter', x='High', y='Low', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
dir(full_m15)